In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots/firstHalf')

In [3]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask', '').replace('_mask', '')
    sides = ['lh', 'rh']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [4]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['lh_ACC', 'rh_ACC', 
             'lh_vmpfc','rh_vmpfc',
             'lh_insula','rh_insula', 
             'lh_hippocampus', 'rh_hippocampus', 
             'lh_amygdala', 'rh_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [5]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [6]:
df_current = df_phase2

In [7]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        df_tmp = df_current.loc[(df_current['roi_name'] == roi) &
                                (df_current['group'] == group) &
                                (df_current['Contrast'].isin([12,14]))].copy()
        df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
        t_statistic, p_value = stats.ttest_rel(df_pivot[[12]], df_pivot[[14]])
        results.append({
            'roi': roi,
            'group': group,
            't_stat': t_statistic[0],
            'p_value': p_value[0],
            'n': len(df_pivot)  # Sample size
        })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,group,t_stat,p_value,n,p_value_corrected
0,lh_hippocampus,Patients,-0.420299,0.676142,49,1.0
17,lh_vmpfc,Controls,0.190886,0.849373,52,1.0
16,lh_vmpfc,Patients,0.202817,0.840135,49,1.0
15,rh_insula,Controls,-0.027748,0.977972,52,1.0
14,rh_insula,Patients,-0.279302,0.781213,49,1.0
13,rh_amygdala,Controls,-1.835810,0.072219,52,1.0
12,rh_amygdala,Patients,-1.030135,0.308110,49,1.0
11,lh_insula,Controls,0.424397,0.673061,52,1.0
10,lh_insula,Patients,0.867628,0.389915,49,1.0
9,rh_vmpfc,Controls,0.493967,0.623450,52,1.0


In [20]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin([12,14]))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[12]], df_pivot[[14]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
52,rh_amygdala,Controls,Placebo,Female,-3.811153,0.002161,14,0.172886
0,lh_hippocampus,Patients,Placebo,Female,-0.444462,0.664619,13,1.000000
57,rh_insula,Patients,Oxytocin,Female,0.435635,0.669305,16,1.000000
56,rh_insula,Patients,Placebo,Female,-0.226041,0.824974,13,1.000000
55,rh_amygdala,Controls,Oxytocin,Male,0.660579,0.522476,12,1.000000
...,...,...,...,...,...,...,...,...
23,lh_amygdala,Controls,Oxytocin,Male,0.382777,0.709178,12,1.000000
22,lh_amygdala,Controls,Placebo,Male,-1.003325,0.334015,14,1.000000
21,lh_amygdala,Controls,Oxytocin,Female,-0.707909,0.493729,12,1.000000
78,rh_hippocampus,Controls,Placebo,Male,0.621668,0.544909,14,1.000000


In [21]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([12,14]))].copy()
    df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.8266
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.593
Time:                        11:35:00   Log-Likelihood:                -139.34
No. Observations:                 101   AIC:                             298.7
Df Residuals:                      91   BIC:                             324.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
29,lh_amygdala,demo_age,0.003197
12,rh_ACC,C(Drug)[T.Placebo],0.024012
14,rh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo],0.032994
25,lh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],0.037567
64,rh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.046258
...,...,...,...
61,rh_amygdala,C(group)[T.Patients],0.961632
77,rh_insula,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.980153
3,lh_hippocampus,C(Gender)[T.Male],0.988093
91,rh_hippocampus,C(group)[T.Patients],0.990643


for the entire dataset \
drug and drug * group in rh-ACC \
group * gender, group * drug, and drug * gender in lh-amygdala \
age in lh-amy and lh-hip

In [10]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([12,14]))].copy()
    df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                 -0.051
Method:                 Least Squares   F-statistic:                    0.5284
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.753
Time:                        11:31:32   Log-Likelihood:                -69.489
No. Observations:                  50   AIC:                             151.0
Df Residuals:                      44   BIC:                             162.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
37,rh_amygdala,C(group)[T.Patients],-0.961865,0.399965,-2.404874,0.020452,-1.767941,-0.155789
39,rh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],1.171380,0.567228,2.065096,0.044837,0.028207,2.314552
57,rh_hippocampus,C(group)[T.Patients]:C(Gender)[T.Male],0.989925,0.600557,1.648345,0.106405,-0.220418,2.200268
56,rh_hippocampus,C(Gender)[T.Male],-0.652958,0.398148,-1.639989,0.108138,-1.455372,0.149456
47,rh_insula,demo_age,0.034707,0.021971,1.579719,0.121334,-0.009571,0.078986
38,rh_amygdala,C(Gender)[T.Male],-0.562504,0.376052,-1.495816,0.141840,-1.320387,0.195378
55,rh_hippocampus,C(group)[T.Patients],-0.624450,0.423466,-1.474618,0.147435,-1.477890,0.228989
28,rh_vmpfc,Avoidance_centered,-0.010596,0.007348,-1.441940,0.156400,-0.025406,0.004214
8,rh_ACC,C(Gender)[T.Male],0.549190,0.400262,1.372077,0.176995,-0.257484,1.355864
16,lh_amygdala,Avoidance_centered,-0.009034,0.007258,-1.244702,0.219830,-0.023662,0.005593


In placebo. \
group and group * gender: rh-amy

In [12]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([12,14]))].copy()
df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [65]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([12,14]))].copy()
df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [13]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([12,14]))].copy()
df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [14]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin([12,14]))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.6924
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.632
Time:                        11:32:50   Log-Likelihood:                -67.631
No. Observations:                  49   AIC:                             147.3
Df Residuals:                      43   BIC:                             158.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

,Condition,Effect,p_value
77,lh_amygdala_Controls,demo_age,0.006318
119,rh_hippocampus_Controls,demo_age,0.031551
101,rh_amygdala_Controls,demo_age,0.043542
67,rh_ACC_Controls,C(Drug)[T.Placebo],0.051121
65,lh_hippocampus_Controls,demo_age,0.051134
...,...,...,...
62,lh_hippocampus_Controls,C(Gender)[T.Male],0.967502
91,lh_insula_Controls,C(Drug)[T.Placebo],0.971641
26,rh_vmpfc_Patients,C(Gender)[T.Male],0.984615
116,rh_hippocampus_Controls,C(Gender)[T.Male],0.990863


in group specific dataset, \
for controls: \
age: bi-amy; rh-hip\
drug (marginal): rh-acc \
age * gender (marginal): lh-amy

In [22]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin([12, 14]))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == 12, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == 14, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'group': group,
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [23]:
df_t_test

,group,ROI,Comparison,Mean_Diff,SE,t,p
22,Controls,rh_ACC,Female_Oxytocin vs. Female_Placebo,0.768769,0.366181,2.130032,0.043621
12,Patients,rh_amygdala,Female_Oxytocin vs. Female_Placebo,0.681755,0.412605,1.643250,0.111928
38,Controls,rh_hippocampus,Female_Oxytocin vs. Female_Placebo,-0.537147,0.354813,-1.513998,0.143086
10,Patients,lh_insula,Female_Oxytocin vs. Female_Placebo,-0.598347,0.401958,-1.479449,0.150594
32,Controls,rh_amygdala,Female_Oxytocin vs. Female_Placebo,-0.466205,0.332741,-1.423149,0.167563
27,Controls,lh_ACC,Male_Oxytocin vs. Male_Placebo,-0.539447,0.381703,-1.414914,0.169938
9,Patients,rh_vmpfc,Male_Oxytocin vs. Male_Placebo,-0.535240,0.400288,-1.406372,0.176643
3,Patients,rh_ACC,Male_Oxytocin vs. Male_Placebo,-0.537807,0.480163,-1.137526,0.270235
0,Patients,lh_hippocampus,Female_Oxytocin vs. Female_Placebo,-0.444725,0.457614,-1.042697,0.306335
24,Controls,lh_amygdala,Female_Oxytocin vs. Female_Placebo,0.464546,0.469088,1.007248,0.323863


In [17]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin([12,14]))].copy()
        df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.8809
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.501
Time:                        11:33:13   Log-Likelihood:                -75.675
No. Observations:                  55   AIC:                             163.3
Df Residuals:                      49   BIC:                             175.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

,Condition,Gender,Effect,p_value
8,rh_ACC_Female,Female,C(Drug)[T.Placebo],0.012835
9,rh_ACC_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.015175
17,lh_amygdala_Female,Female,demo_age,0.016170
7,rh_ACC_Female,Female,C(group)[T.Patients],0.046205
39,rh_amygdala_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.050840
...,...,...,...,...
37,rh_amygdala_Female,Female,C(group)[T.Patients],0.987177
82,lh_ACC_Male,Male,Avoidance_centered,0.988536
115,rh_hippocampus_Male,Male,C(group)[T.Patients],0.989636
44,rh_insula_Female,Female,C(Drug)[T.Placebo],0.993866


in gender specific dataset:\
For female:\
drug, group and group * drug: rh-acc\
group * drug: rh-amy\
age: lh-amy\
For male:\
group in rh-amy


In [18]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin([12,14]))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==12, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==14, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     1.183
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.336
Time:                        11:33:24   Log-Likelihood:                -39.225
No. Observations:                  29   AIC:                             86.45
Df Residuals:                      25   BIC:                             91.92
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
159,Male,Controls,rh_hippocampus_Controls_Male,demo_age,0.015152
154,Male,Controls,lh_vmpfc_Controls_Male,Avoidance_centered,0.023567
131,Male,Controls,lh_amygdala_Controls_Male,demo_age,0.028532
45,Female,Controls,rh_ACC_Controls_Female,C(Drug)[T.Placebo],0.043080
138,Male,Controls,rh_vmpfc_Controls_Male,Avoidance_centered,0.070825
...,...,...,...,...,...
61,Female,Controls,lh_insula_Controls_Female,C(Drug)[T.Placebo],0.979415
91,Male,Patients,lh_amygdala_Patients_Male,demo_age,0.981247
18,Female,Patients,rh_vmpfc_Patients_Female,Avoidance_centered,0.986655
113,Male,Patients,lh_vmpfc_Patients_Male,C(Drug)[T.Placebo],0.991405


in group * gender specific dataset:\
For male cotrols:\
age in rh-hip, lh-amy\
avoidance in lh-vmpfc (rh-vmpfc, marginal)\
For female controls:
drug in rh-acc


In [19]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([12, 14]))].copy()
    df_tmp.loc[df_tmp['Contrast'] == 12, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == 14, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/extinction_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

lh_hippocampus
rh_ACC
lh_amygdala
lh_ACC
rh_vmpfc
lh_insula
rh_amygdala
rh_insula
lh_vmpfc
rh_hippocampus
